The purpose of this code is to get a quick summary of potential ROIs, as determined by a whole brain cFos mapping pilot. In this case, individual csv files we are pulling from were provided by LifeCanvas Technologies

This code will:
1. Create a dataframe combining all cFos files in a given folder, with new columns specifying (1) subject's experimental condition and (2) unique subject ID
2. Create an output file of plots summarizing data from each brain region, where each brain region has its own row. In each row:
- The left plot with contain cFos densities from left hemisphere, separated by experimental condition
- The middle plot will contain cFos densities from the right hemisphere, separated by experimental condition
- The right plot will contain average cFos densities from combined hemispheres
3. Create an output file of p values, ranked in descending order of statistical significance, that can be used to search for potential plots of interest

In [1]:
#IMPORT REQUIRED LIBRARIES
import pandas as pd
import glob
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages
import scipy.stats as stats

In [2]:
#SPECIFY PATH TO FOLDER CONTAINING DATA
from google.colab import drive #Mount google drive: if your datasets are not stored in gdrive, skip this
drive.mount('/content/gdrive')
PATH = "gdrive/MyDrive/Whole brain cFos analysis/Dam Spreadsheets/"

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [3]:
#COMBINE DATA FROM ALL DATA FILES INTO SINGLE DATA FRAME

# Get list of CSV files in path
file_paths = glob.glob(PATH + '*.csv')

# Initialize an empty list to store individual DataFrames
dfs = []

#Create individual DataFrames for each CSV file
for i in range(0,len(file_paths)):
    df = pd.read_csv(file_paths[i])
    # Add a columns for 1) the unique mom ID and 2) condition (sep or no sep)
    filePath = file_paths[i] #using specific file name
    index = filePath.find("Mom") #find "Mom", which preceeds unique dam ID
    if index != -1 and index + len("Mom") < len(filePath): #get index of number after "Mom"
      DamID = filePath[index + len("Mom")]
      df["DamID"] = DamID #append column with Dam ID
    if index != -1 and index + len("Mom") < len(filePath): #get index of condition before ".csv"
      DamID = filePath[index + (len("Mom")+2)]
      df["Condition"] = DamID #append column with Condition
    # Append the DataFrame to the list
    dfs.append(df)

# Concatenate all DataFrames in the list vertically
combined_df = pd.concat(dfs, ignore_index=True)

# Print the combined DataFrame
combined_df

,id,name,acronym,parent_structure_id,depth,count,volume (mm^3),density (cells/mm^3),DamID,Condition
0,-1,background,bkd,-2.0,-1.0,12287.0,699.556516,17.563985,1,n
1,0,left root,root-L,-1.0,0.0,67134.0,250.985562,267.481521,1,n
2,1,left Basic cell groups and regions,grey-L,0.0,1.0,65136.0,224.592938,290.018024,1,n
3,2,left Cerebrum,CH-L,1.0,2.0,57995.0,137.893719,420.577533,1,n
4,3,left Cerebral cortex,CTX-L,2.0,3.0,55589.0,110.753453,501.916630,1,n
...,...,...,...,...,...,...,...,...,...,...
13419,11299,right third ventricle,V3-R,11292.0,2.0,779.0,0.466687,1669.211196,9,S
13420,11300,right cerebral aqueduct,AQ-R,11292.0,2.0,107.0,0.209484,510.777952,9,S
13421,11301,right fourth ventricle,V4-R,11292.0,2.0,158.0,0.485563,325.395804,9,S
13422,11302,right lateral recess,V4r-R,11301.0,3.0,84.0,0.231766,362.435111,9,S


In [4]:
#FIND LIST OF ACRONYMS COMMON TO BOTH HEMISPHERES TO ITERATE THROUGH FOR PLOTTING/ANALYSIS

# Left
left_hemisphere = combined_df[combined_df['acronym'].str.endswith('-L')] #generate dataframe for left hemisphere
unique_acronymsL = list(left_hemisphere['acronym'].unique()) #get one set of left hemisphere acronyms (not repeating by subject)

# Right
right_hemisphere = combined_df[combined_df['acronym'].str.endswith('-R')] #generate dataframe for right hemisphere
unique_acronymsR = list(right_hemisphere['acronym'].unique()) #get one set of right hemisphere acronyms (not repeating by subject)

#Filter out acronyms that don't exist in both hemispheres
predicted_unique_acronymsR = np.core.defchararray.replace(unique_acronymsL, '-L', '-R') #first predict -R acronyms from -L acronyms

#second, filter out predicted -R acronyms not in actual -R acronyms set
final_acronyms = []
for i in range(0,len(predicted_unique_acronymsR)):
  if predicted_unique_acronymsR[i] == unique_acronymsR[i]:
    final_acronyms.append(predicted_unique_acronymsR[i])

#here are final acronyms to use for iteration
final_acronyms = np.core.defchararray.replace(final_acronyms, '-R', '')
final_acronymsR = np.char.add(final_acronyms, '-R')
final_acronymsL = np.char.add(final_acronyms, '-L')

In [5]:
#GENERATE OUTPUT FILE CONTAINING PLOTS FOR ALL REGIONS

# Specify x-axis locations for L v R hemisphere plots
xS = np.zeros(4)  # for separated condition
xN = np.ones(4)  # for non-separated condition

# Plot hemisphere data
# Set up the figure and axes for the subplots
fig, axes = plt.subplots(len(final_acronymsL), 3, sharey='row')
fig.set_size_inches(3, 654)
fig.tight_layout(rect=[0.1, 0, 1, 1])

for i, (acronymL, acronymR) in enumerate(zip(final_acronymsL, final_acronymsR)):
    left_condition_S = (left_hemisphere['acronym'] == acronymL) & (left_hemisphere['Condition'] == 'S')
    left_condition_N = (left_hemisphere['acronym'] == acronymL) & (left_hemisphere['Condition'] == 'n')
    right_condition_S = (right_hemisphere['acronym'] == acronymR) & (right_hemisphere['Condition'] == 'S')
    right_condition_N = (right_hemisphere['acronym'] == acronymR) & (right_hemisphere['Condition'] == 'n')

    # Left hemisphere, both conditions
    leftData_Sep = left_hemisphere.loc[left_condition_S, 'density (cells/mm^3)']
    leftData_No = left_hemisphere.loc[left_condition_N, 'density (cells/mm^3)']
    axes[i, 0].scatter(xS, leftData_Sep)
    axes[i, 0].scatter(xN, leftData_No)
    axes[i, 0].set_title(str(acronymL))
    axes[i, 0].set_xticks([0, 1])
    axes[i, 0].set_xticklabels(['sep', 'no'])

    # Right hemisphere, both conditions
    rightData_Sep = right_hemisphere.loc[right_condition_S, 'density (cells/mm^3)']
    rightData_No = right_hemisphere.loc[right_condition_N, 'density (cells/mm^3)']
    axes[i, 1].scatter(xS, rightData_Sep)
    axes[i, 1].scatter(xN, rightData_No)
    axes[i, 1].set_title(str(acronymR))
    axes[i, 1].set_xticks([0, 1])
    axes[i, 1].set_xticklabels(['sep', 'no'])

    # Average of left and right hemispheres
    avg_densitySep = (leftData_Sep.values + rightData_Sep.values) / 2
    avg_densityNo = (leftData_No.values + rightData_No.values) / 2

    # Plot this average
    axes[i, 2].scatter(xS, avg_densitySep)
    axes[i, 2].scatter(xN, avg_densityNo)
    axes[i, 2].set_title("Average")
    axes[i, 2].set_xticks([0, 1])
    axes[i, 2].set_xticklabels(['sep', 'no'])

# plt.show()
plt.savefig(PATH + 'density_plots.pdf')
plt.close()

In [8]:
#GENERATE OUTPUT FILE CONTAINING STATS FOR ALL REGIONS

#Specify future columns of stat output dataframe
left_pval = []
left_ac = []
right_pval = []
right_ac = []
avg_pval = []
avg = []


#Run t tests on subsets
for i, (acronymL, acronymR) in enumerate(zip(final_acronymsL, final_acronymsR)):
    left_condition_S = (left_hemisphere['acronym'] == acronymL) & (left_hemisphere['Condition'] == 'S')
    left_condition_N = (left_hemisphere['acronym'] == acronymL) & (left_hemisphere['Condition'] == 'n')
    right_condition_S = (right_hemisphere['acronym'] == acronymR) & (right_hemisphere['Condition'] == 'S')
    right_condition_N = (right_hemisphere['acronym'] == acronymR) & (right_hemisphere['Condition'] == 'n')

    # Left hemisphere, both conditions
    leftData_Sep = left_hemisphere.loc[left_condition_S, 'density (cells/mm^3)']  #Obtain Sep, left hemisphere dataframe's density
    leftData_No = left_hemisphere.loc[left_condition_N, 'density (cells/mm^3)']  #Obtain NoSep, left hemisphere dataframe's density
    t_stat_left, p_val_left = stats.ttest_ind(leftData_Sep,leftData_No,equal_var=True) #stat test
    left_pval.append(round(p_val_left,4)) #append result to p_val_left column
    left_ac.append(str(acronymL))

    # Right hemisphere, both conditions
    rightData_Sep = right_hemisphere.loc[right_condition_S, 'density (cells/mm^3)']  #Obtain Sep, right hemisphere dataframe's density
    rightData_No = right_hemisphere.loc[right_condition_N, 'density (cells/mm^3)'] #Obtain NoSep, right hemisphere dataframe's density
    t_stat_right, p_val_right = stats.ttest_ind(rightData_Sep,rightData_No,equal_var=True) #stat test
    right_pval.append(round(p_val_right,4)) #appendresult to p_val_right column
    right_ac.append(str(acronymR))

    # Average of left and right hemispheres
    avg_densitySep = (leftData_Sep.values + rightData_Sep.values) / 2 #Obtain Sep, average density between hemispheres
    avg_densityNo = (leftData_No.values + rightData_No.values) / 2 #Obtain NoSep, average density between hemispheres
    t_stat, p_val = stats.ttest_ind(avg_densitySep,avg_densityNo,equal_var=True) #stat test
    avg_pval.append(round(p_val,4))
    avg.append("Average")

#It will also be helpful to have full names
dictionary = {key: value for key, value in zip(left_hemisphere['acronym'], left_hemisphere['name'])} #making dictionary to retrieve full name info
names = []
for z in range(0,len(left_ac)):
  names.append(dictionary[str(left_ac[z])])
names = [' '.join(entry.split()[1:]) for entry in names] #get rid of 'left'



#Make summary dataframe, ranked by statistical significance
pVals = pd.DataFrame({'pVal-L':left_pval,'acronym-L':left_ac,'pVal-R':right_pval,'acronym-R':right_ac,'pVal-T':avg_pval,'Whole':avg,'Names':names})
pVals_sorted = pVals.sort_values(by='pVal-T')

#Save to path
# Initialize the PDF pages
matplotlib.use('Agg') # prevent preview of plot from appearing here
with PdfPages(PATH + 'stats.pdf') as pdf:
    # Create a new page
    fig, ax = plt.subplots(figsize=(14, 20))

    # Set up table properties
    table = ax.table(cellText=pVals_sorted.values, colLabels=pVals_sorted.columns, cellLoc='center', loc='center')
    table.auto_set_font_size(False)
    table.set_fontsize(12)
    table.scale(1.2, 1.2)

    # Remove axis
    ax.axis('off')

    # Auto adjust the column widths
    table.auto_set_column_width(col=list(range(len(pVals_sorted.columns))))

    # Add the table to the PDF page
    pdf.savefig(fig, bbox_inches='tight')


In [14]:
# Create dictionary from columns
dictionary = {key: value for key, value in zip(left_hemisphere['acronym'], left_hemisphere['name'])}
print(dictionary)

{'root-L': 'left root', 'grey-L': 'left Basic cell groups and regions', 'CH-L': 'left Cerebrum', 'CTX-L': 'left Cerebral cortex', 'CTXpl-L': 'left Cortical plate', 'Isocortex-L': 'left Isocortex', 'FRP-L': 'left Frontal pole, cerebral cortex', 'FRP1-L': 'left Frontal pole, layer 1', 'FRP2/3-L': 'left Frontal pole, layer 2/3', 'FRP5-L': 'left Frontal pole, layer 5', 'FRP6a-L': 'left Frontal pole, layer 6a', 'FRP6b-L': 'left Frontal pole, layer 6b', 'MO-L': 'left Somatomotor areas', 'MOp-L': 'left Primary motor area', 'MOp1-L': 'left Primary motor area, Layer 1', 'MOp2/3-L': 'left Primary motor area, Layer 2/3', 'MOp5-L': 'left Primary motor area, Layer 5', 'MOp6a-L': 'left Primary motor area, Layer 6a', 'MOp6b-L': 'left Primary motor area, Layer 6b', 'MOs-L': 'left Secondary motor area', 'MOs1-L': 'left Secondary motor area, layer 1', 'MOs2/3-L': 'left Secondary motor area, layer 2/3', 'MOs5-L': 'left Secondary motor area, layer 5', 'MOs6a-L': 'left Secondary motor area, layer 6a', 'MOs

In [17]:
left_ac[1]

'grey-L'

In [18]:
dictionary = {key: value for key, value in zip(left_hemisphere['acronym'], left_hemisphere['name'])} #making dictionary to retrieve full name info
names = []
for z in range(0,len(left_ac)):
  names.append(dictionary[str(left_ac[z])])
names = [' '.join(entry.split()[1:]) for entry in names] #get rid of 'left'

In [21]:
type(names)

list

In [22]:
len(names)

796

In [23]:
names = [' '.join(entry.split()[1:]) for entry in names]
print(new_list)

['root', 'Basic cell groups and regions', 'Cerebrum', 'Cerebral cortex', 'Cortical plate', 'Isocortex', 'Frontal pole, cerebral cortex', 'Frontal pole, layer 1', 'Frontal pole, layer 2/3', 'Frontal pole, layer 5', 'Frontal pole, layer 6a', 'Frontal pole, layer 6b', 'Somatomotor areas', 'Primary motor area', 'Primary motor area, Layer 1', 'Primary motor area, Layer 2/3', 'Primary motor area, Layer 5', 'Primary motor area, Layer 6a', 'Primary motor area, Layer 6b', 'Secondary motor area', 'Secondary motor area, layer 1', 'Secondary motor area, layer 2/3', 'Secondary motor area, layer 5', 'Secondary motor area, layer 6a', 'Secondary motor area, layer 6b', 'Somatosensory areas', 'Primary somatosensory area', 'Primary somatosensory area, nose', 'Primary somatosensory area, nose, layer 1', 'Primary somatosensory area, nose, layer 2/3', 'Primary somatosensory area, nose, layer 4', 'Primary somatosensory area, nose, layer 5', 'Primary somatosensory area, nose, layer 6a', 'Primary somatosensory